## Environment setup

In [1]:
!python3.7 -m pip install boto3;

     |████████████████████████████████| 133kB 21.2MB/s eta 0:00:01


In [2]:
!python3.7 -m pip install awscli;

## Parameters configuration

In [3]:
import boto3

account = boto3.client('sts').get_caller_identity().get('Account')

AWS_REGION = "us-east-1"

# Configuration for the IoT Thing: Certificate file name, and core name
CERT_PEM_OUTFILE="mli.cert.pem"
PUBLIC_KEY_OUTFILE="mli.public.key" 
PRIVATE_KEY_OUTFILE="mli.private.key" 
CORENAME="greengrass_ml_{}".format(AWS_REGION)

# Temporary workspace
WORKSPACE_FOLDER="./work"

# S3 Bucket for the ML model
ML_S3_BUCKET="{}-greengrass-{}".format(account,AWS_REGION)

# S3 Bucket for the Cloudformation functions
CFN_S3_BUCKET="{}-cloudformation-{}".format(account,AWS_REGION)
CFN_STACK_NAME="greengrass-mli-accelerator"

In [4]:
![ -d {WORKSPACE_FOLDER} ] && rm -r {WORKSPACE_FOLDER} && echo "old {WORKSPACE_FOLDER} removed"
!mkdir -p {WORKSPACE_FOLDER} && echo "{WORKSPACE_FOLDER} created"

old ./work removed
./work created


## Create the credential for the Greengrass Core

In [5]:
iotClient = boto3.client('iot', region_name=AWS_REGION)
from botocore.exceptions import ClientError
try:
    response = iotClient.create_keys_and_certificate(
        setAsActive=True
    )
except ClientError as e:
    if(e.response["Error"]["Code"]=="AccessDeniedException"):
        print("Missing permission. Please add \niot:CreateKeysAndCertificate on resource: *\n to the instance IAM role\n")
    raise e

In [6]:
import os
CERTIFICATE_ID=response.get("certificateId")

try:
    with open(os.path.join(WORKSPACE_FOLDER, CERT_PEM_OUTFILE), 'w') as the_file:
        the_file.write(response.get("certificatePem"))
    with open(os.path.join(WORKSPACE_FOLDER, PUBLIC_KEY_OUTFILE), 'w') as the_file:
        the_file.write(response.get("keyPair").get("PublicKey"))
    with open(os.path.join(WORKSPACE_FOLDER, PRIVATE_KEY_OUTFILE), 'w') as the_file:
        the_file.write(response.get("keyPair").get("PrivateKey"))
except IOError as e:
    print("Error creating certificate files")
    raise e

## Download the pre-trained model

In [7]:
!wget -nv http://data.mxnet.io/models/imagenet/inception-bn/Inception-BN-symbol.json --directory-prefix {WORKSPACE_FOLDER}
!wget -nv http://data.mxnet.io/mxnet/models/imagenet/synset.txt --directory-prefix {WORKSPACE_FOLDER}
!wget -nv http://data.mxnet.io/models/imagenet/inception-bn/Inception-BN-0126.params -O {WORKSPACE_FOLDER}/Inception-BN-0000.params

2019-11-09 20:05:09 URL:http://data.mxnet.io.s3-website-us-west-1.amazonaws.com/models/imagenet/inception-bn/Inception-BN-symbol.json [116922/116922] -> "./work/Inception-BN-symbol.json" [1]
2019-11-09 20:05:09 URL:http://data.mxnet.io.s3-website-us-west-1.amazonaws.com/mxnet/models/imagenet/synset.txt [31675/31675] -> "./work/synset.txt" [1]
2019-11-09 20:05:12 URL:http://data.mxnet.io.s3-website-us-west-1.amazonaws.com/models/imagenet/inception-bn/Inception-BN-0126.params [45284780/45284780] -> "./work/Inception-BN-0000.params" [1]


In [8]:
from zipfile import ZipFile

# create a ZipFile object
zipObj = ZipFile(os.path.join(WORKSPACE_FOLDER, "inception-bn.zip"), 'w')
# Add multiple files to the zip
zipObj.write(os.path.join(WORKSPACE_FOLDER,"Inception-BN-symbol.json"))
zipObj.write(os.path.join(WORKSPACE_FOLDER,"synset.txt"))
zipObj.write(os.path.join(WORKSPACE_FOLDER,"Inception-BN-0000.params"))
# close the Zip File
zipObj.close()

In [9]:
# Upload to S3 bucket
import boto3
import os
s3 = boto3.resource('s3')
if not s3.Bucket(ML_S3_BUCKET) in s3.buckets.all():
    s3.create_bucket(Bucket=ML_S3_BUCKET)

s3_client = boto3.client('s3')
s3_client.upload_file( os.path.join(WORKSPACE_FOLDER, "inception-bn.zip"), ML_S3_BUCKET, "inception-bn.zip")

ML_S3_BUCKET_URI="s3://{}/{}".format(ML_S3_BUCKET, "inception-bn.zip")

## Uses the Cloudformation to generate the Greengrass Core

In [16]:
!git clone https://github.com/awslabs/aws-iot-greengrass-accelerators.git

Cloning into 'aws-iot-greengrass-accelerators'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 208 (delta 67), reused 170 (delta 43), pack-reused 0
Receiving objects: 100% (208/208), 5.34 MiB | 19.11 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [10]:
import boto3
s3 = boto3.resource('s3')
if not s3.Bucket(CFN_S3_BUCKET) in s3.buckets.all():
    s3.create_bucket(Bucket=CFN_S3_BUCKET)

In [11]:
# Check IAM permission
import boto3
from botocore.exceptions import ClientError

try:
    cloudformation = boto3.resource('cloudformation', region_name=AWS_REGION)
    stack = cloudformation.Stack(CFN_STACK_NAME)
    stack.description
except ClientError as e:
    if(e.response["Error"]["Code"]=="ValidationError"):
        print("Stack name {} does not exist, continue".format(CFN_STACK_NAME))
    elif(e.response["Error"]["Code"]=="AccessDenied"):
        print("Missing permission. Please add the following IAM Policy\n\n \
              resource: arn:aws:cloudformation:{}:{}:stack/{}/*\n\n \
              cloudformation:DescribeStacks \n \
              cloudformation:CreateChangeSet \n \
              to IAM role:\n{}\n".format(region,account,CFN_STACK_NAME,role))
        raise e
    else:
        raise e

Stack name greengrass-mli-accelerator does not exist, continue


In [18]:
![ -e {WORKSPACE_FOLDER}/*-OUTPUT.yaml ] && rm {WORKSPACE_FOLDER}/*-OUTPUT.yaml
!aws cloudformation package \
--region {AWS_REGION} \
--template-file aws-iot-greengrass-accelerators/accelerators/machine_learning_inference/cfn/mli_accelerator_s3_models-INPUT.cfn.yaml \
--s3-bucket {CFN_S3_BUCKET} \
--output-template-file {WORKSPACE_FOLDER}/mli_accelerator_s3_models-OUTPUT.yaml


Uploading to c232aa669d7875a2f0a0e474cdfa9208  3751 / 3751.0  (100.00%)00%)
Successfully packaged artifacts and wrote output template to file ./work/mli_accelerator_s3_models-OUTPUT.yaml.
Execute the following command to deploy the packaged template
aws cloudformation deploy --template-file /home/jovyan/work/mli_accelerator_s3_models-OUTPUT.yaml --stack-name <YOUR STACK NAME>


In [19]:
!aws cloudformation deploy \
  --region {AWS_REGION} \
  --stack-name {CFN_STACK_NAME} \
  --template-file {WORKSPACE_FOLDER}/mli_accelerator_s3_models-OUTPUT.yaml \
  --capabilities CAPABILITY_NAMED_IAM \
  --parameter-overrides \
    CoreName={CORENAME} \
    CertIdParam={CERTIFICATE_ID} \
    ModelS3Uri={ML_S3_BUCKET_URI} 


Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - greengrass-mli-accelerator


In [21]:
print("...waiting for stack {} to be ready...".format(CFN_STACK_NAME))
client = boto3.client('cloudformation', region_name=AWS_REGION)
waiter = client.get_waiter('stack_create_complete')
waiter.wait(StackName=CFN_STACK_NAME)

...waiting for stack greengrass-mli-accelerator to be ready...


## Package the credentials and config